In [1]:
import pandas as pd

In [2]:
import tweepy,twitter

In [56]:
auth = tweepy.OAuthHandler('D4cA5heqwzCJryH1GnUzEDopn', 'wjSCGp8t4uP3M1egnzy8lhmja3JR04Pw2B2D01gZHz0Kf6iSvt')
auth.set_access_token('823928942084005889-Js2sE3uVV6ETxsFwo6izUanxhk0V1VB', 'Q3WvYiDK726YmGKWxvke4zaFQQvYtKOxu8cMuwT7twjPO')

api = tweepy.API(auth)

In [4]:
suicidal_tweets=pd.read_csv("suicidal_data.csv")

In [5]:
import pickle
import re
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor


In [7]:
def preprocess_tweet(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text+' '.join(emoticons).replace('-', '') 
    return text

In [8]:
tqdm.pandas()
suicidal_tweets['tweet'] = suicidal_tweets['tweet'].progress_apply(preprocess_tweet)

C:\Users\user\Desktop\sample_project\env\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████| 9119/9119 [00:00<00:00, 15287.35it/s]


In [9]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [11]:
[w for w in tokenizer_porter('a swimmer likes swimming and swims a lot') if w not in stop]

['swimmer', 'like', 'swim', 'swim', 'lot']

In [12]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text += ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in tokenizer_porter(text) if w not in stop]
    return tokenized

## Using the Hashing Vectorizer

In [13]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None,tokenizer=tokenizer)

## Building the ml model

In [14]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1)

In [15]:
X = suicidal_tweets["tweet"].to_list()
y = suicidal_tweets['label']

## Training the model

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [18]:
classes = np.array([0, 1])
clf.partial_fit(X_train, y_train,classes=classes)

SGDClassifier(loss='log', random_state=1)

In [19]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.912


In [20]:
clf = clf.partial_fit(X_test, y_test)

## Testing 

In [21]:
label = {0:'negative', 1:'positive'}

In [22]:
phrases = ["suicide","suicidal", "kill myself", "my suicide note", "my suicide letter", "end my life", "never wake up", "can't go on"," cannot go on", "not worth living", "ready to jump", "swleep forever", "want to die", "be dead", "better off without me", "better off dead", "don't want to being here", "tired of living", "die alone", "go to sleep forever", "wanna die", "wanna suicide", "commit suicide", "slit my wrist","cut my wrist", "slash my wrist", "do not want to be here", "want it to be over", "want to be dead", "nothing to live for", "not worth living","ready to die", "thoughts of suicide", "thoughts of killing myself", "why should i live", "take my own life", "depressed"]

In [75]:
streamed_tweets=[]
for query in phrases:   
    tweets = api.search(q= query + " -filter:retweets", lang="en", show_user=True, result_type="recent")
    for i in tweets:
        print(i.text)
        streamed_tweets.append({'tweet_id': i.id,'tweet_text': i.text, 'user_name': i.user.screen_name})

Henry Ford right before committing suicide: "Henry Ford more like Henry Fjord" (and then he throws himself off a fjord).
@crazygamerofSA @PS5Status @PS5Countdowns I mean well of course? I thought we all knew we wouldn’t see full next ge… https://t.co/XS1ObUuogE
THE World Health Organisation (WHO) sometimes revealed that Nigeria has the highest suicidal rate among African cou… https://t.co/aF2lbbbg1k
Suicide? //

Ik everyone fucking hatez me i hope i dont fucking wake uo
@TreeBoi163 @MethaTheRet @cat_marinara @Bigotedhorse @ImPieFreak @Twitch Lmfao you guys are still missing the point… https://t.co/0SYgNwCYeL
Consider Suicide
Suicide ain’t a joke, watch your loved ones extra closely
Man Commits Suicide After Killing 17-yr-old Girl in UP's Banda https://t.co/0yDrHC5fAP
Kids and adult mental health is suffering beyond any effects of a pandemic. Anxiety. Stress. Suicide rates. Focus o… https://t.co/xpLIts2UOm
@HotBoyMK_ she's gay and the woman is abusive :/ I read the thread and she be usi

In [76]:
streamed_tweet_df=pd.DataFrame.from_dict(streamed_tweets)
streamed_tweet_df

,tweet_id,tweet_text,user_name
0,1330417443156574208,"Henry Ford right before committing suicide: ""H...",ben_jammin117
1,1330417443030904832,@crazygamerofSA @PS5Status @PS5Countdowns I me...,SykeRoze
2,1330417433698578438,THE World Health Organisation (WHO) sometimes ...,KaylebsThought
3,1330417415189114880,Suicide? //\n\nIk everyone fucking hatez me i ...,kishikujo
4,1330417413645623297,@TreeBoi163 @MethaTheRet @cat_marinara @Bigote...,Mysthiccc
...,...,...,...
525,1330417241930784768,what? im not depressed! *profusely sweating*,ballerinarob
526,1330417206132428802,A bitch not even depressed just lonely like fu...,mvriexvo
527,1330417190269575170,@ShanlinClaire Chillll I’m already depressed.,HollandDaltonn
528,1330417175937626118,depressed as always.. https://t.co/iTidxckNQ3,maisamAKAmunna


In [77]:
analysis= pd.DataFrame(columns={"Prediction","Probability"})
analysis["Prediction"].astype(object)
analysis["Probability"].astype(float)

Series([], Name: Probability, dtype: float64)

In [78]:
for row in range(0,len(streamed_tweet_df)):
    example=[streamed_tweet_df.iloc[row,1]]
    X = vect.transform(example)
    print('Prediction: %s   Probability: %.2f%%'
    %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))
    analysis=analysis.append({'Prediction': label[clf.predict(X)[0]],
                                             'Probability': np.max(clf.predict_proba(X))*100},
                                            ignore_index=True)

Prediction: negative   Probability: 70.28%
Prediction: negative   Probability: 91.90%
Prediction: negative   Probability: 57.94%
Prediction: positive   Probability: 91.63%
Prediction: negative   Probability: 94.23%
Prediction: positive   Probability: 90.86%
Prediction: negative   Probability: 58.38%
Prediction: negative   Probability: 59.04%
Prediction: positive   Probability: 57.38%
Prediction: negative   Probability: 91.43%
Prediction: positive   Probability: 78.90%
Prediction: negative   Probability: 87.74%
Prediction: positive   Probability: 55.46%
Prediction: positive   Probability: 82.80%
Prediction: negative   Probability: 61.28%
Prediction: negative   Probability: 57.94%
Prediction: negative   Probability: 97.31%
Prediction: negative   Probability: 97.34%
Prediction: negative   Probability: 52.74%
Prediction: negative   Probability: 97.45%
Prediction: negative   Probability: 96.82%
Prediction: negative   Probability: 86.16%
Prediction: negative   Probability: 91.31%
Prediction:

In [79]:
merge=streamed_tweet_df.merge(analysis, how="inner", left_index=True, right_index=True)

In [80]:
merge

,tweet_id,tweet_text,user_name,Prediction,Probability
0,1330417443156574208,"Henry Ford right before committing suicide: ""H...",ben_jammin117,negative,70.280242
1,1330417443030904832,@crazygamerofSA @PS5Status @PS5Countdowns I me...,SykeRoze,negative,91.902759
2,1330417433698578438,THE World Health Organisation (WHO) sometimes ...,KaylebsThought,negative,57.941155
3,1330417415189114880,Suicide? //\n\nIk everyone fucking hatez me i ...,kishikujo,positive,91.626704
4,1330417413645623297,@TreeBoi163 @MethaTheRet @cat_marinara @Bigote...,Mysthiccc,negative,94.231527
...,...,...,...,...,...
525,1330417241930784768,what? im not depressed! *profusely sweating*,ballerinarob,negative,84.775141
526,1330417206132428802,A bitch not even depressed just lonely like fu...,mvriexvo,negative,55.816735
527,1330417190269575170,@ShanlinClaire Chillll I’m already depressed.,HollandDaltonn,negative,64.572661
528,1330417175937626118,depressed as always.. https://t.co/iTidxckNQ3,maisamAKAmunna,negative,78.238703


In [81]:
mask = (merge['Probability'].ge(80)) & (merge['Prediction']=='positive')
merge['mask'] = np.where(mask,'Flagged', 'no problem')
merge

,tweet_id,tweet_text,user_name,Prediction,Probability,mask
0,1330417443156574208,"Henry Ford right before committing suicide: ""H...",ben_jammin117,negative,70.280242,no problem
1,1330417443030904832,@crazygamerofSA @PS5Status @PS5Countdowns I me...,SykeRoze,negative,91.902759,no problem
2,1330417433698578438,THE World Health Organisation (WHO) sometimes ...,KaylebsThought,negative,57.941155,no problem
3,1330417415189114880,Suicide? //\n\nIk everyone fucking hatez me i ...,kishikujo,positive,91.626704,Flagged
4,1330417413645623297,@TreeBoi163 @MethaTheRet @cat_marinara @Bigote...,Mysthiccc,negative,94.231527,no problem
...,...,...,...,...,...,...
525,1330417241930784768,what? im not depressed! *profusely sweating*,ballerinarob,negative,84.775141,no problem
526,1330417206132428802,A bitch not even depressed just lonely like fu...,mvriexvo,negative,55.816735,no problem
527,1330417190269575170,@ShanlinClaire Chillll I’m already depressed.,HollandDaltonn,negative,64.572661,no problem
528,1330417175937626118,depressed as always.. https://t.co/iTidxckNQ3,maisamAKAmunna,negative,78.238703,no problem


In [89]:
display=pd.DataFrame(columns=['User Name','Tweet'])

for row in range(len(merge)):
        if merge['mask'][row]=='Flagged': 
            display.loc[row,"User Name"]= merge['user_name'][row] 
            display.loc[row,"Tweet"]= merge['tweet_text'][row]
      

In [90]:
display

,User Name,Tweet
3,kishikujo,Suicide? //\n\nIk everyone fucking hatez me i ...
5,__nlop,Consider Suicide
13,DRACULA_SIAN,tw // depression suicide mention and self harm...
23,Nolali_,The only time I'd call a friend's parent is wh...
26,reveillution,Its like this if shes suicidal which she is wh...
28,idara_mmi,one cant tell you they're depressed and consid...
54,killmyself6666,@jayd3nnn Writing my suicide note
58,Availab1le,This song is my suicide note...
64,dongjunstan,tw // suicide\n\nthey think that i like being ...
71,diptychbitch,in the first suicide letter I ever wrote when ...


In [91]:
def send():
    user_list = []
    for row in range(len(merge)):
        if merge['mask'][row]=='Flagged':
            user_list.append({'handle': merge['user_name'][row], 'tweet_id': merge['tweet_id'][row]})
    return(user_list)

In [92]:
send()

[{'handle': 'kishikujo', 'tweet_id': 1330417415189114880},
 {'handle': '__nlop', 'tweet_id': 1330417372885291008},
 {'handle': 'DRACULA_SIAN', 'tweet_id': 1330417155129679877},
 {'handle': 'Nolali_', 'tweet_id': 1330416340742660096},
 {'handle': 'reveillution', 'tweet_id': 1330415558962057217},
 {'handle': 'idara_mmi', 'tweet_id': 1330415464766431237},
 {'handle': 'killmyself6666', 'tweet_id': 1330361986304139264},
 {'handle': 'Availab1le', 'tweet_id': 1330354754866843648},
 {'handle': 'dongjunstan', 'tweet_id': 1330087165209047040},
 {'handle': 'diptychbitch', 'tweet_id': 1329884437165826049},
 {'handle': 'yungpsychadelic', 'tweet_id': 1330412891128745984},
 {'handle': 'Mattngiwjnsjsjs', 'tweet_id': 1330378665381818370},
 {'handle': 'ahoy_asho', 'tweet_id': 1330359712387391488},
 {'handle': 'Bigmaheadwar', 'tweet_id': 1330411708079362048},
 {'handle': 'KillingThatBoy', 'tweet_id': 1330401215927226369},
 {'handle': 'AlDadson', 'tweet_id': 1330414135834783747},
 {'handle': 'meganenokimi